In [134]:
### Imports
from pathlib import Path
import scipy.sparse as sparse
import implicit
import pandas as pd
import numpy as np

## Load datasets

In [135]:
# Order datasets
df_order_products_prior = pd.read_csv("../data/order_products__prior.csv")
df_orders = pd.read_csv("../data/orders.csv")

In [136]:
df_order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [137]:
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


## Clean data

In [138]:
# Consider only "prior" orders and remove all columns except order_id and user_id from df_orders
prior_order_user = df_orders.loc[df_orders.eval_set == "prior"]
prior_order_user = prior_order_user[["order_id", "user_id"]]

In [139]:
prior_order_user.head()

,order_id,user_id
0,2539329,1
1,2398795,1
2,473747,1
3,2254736,1
4,431534,1


In [140]:
# Remove all columns except order_id and user_id from df_orders
prior_order_product = df_order_products_prior[["order_id", "product_id"]]

In [141]:
prior_order_product.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


## Build user product matrix

In [178]:
merged_order_product_user = pd.merge(prior_order_user, prior_order_product, on="order_id")

In [179]:
merged_order_product_user.head()

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405


In [193]:
df_user_product = merged_order_product_user[["user_id", "product_id"]]

,user_id,product_id
0,1,196
1,1,14084
2,1,12427
3,1,26088
4,1,26405


In [194]:
df_user_product = df_user_product.groupby(["user_id", "product_id"]).size().reset_index().rename(columns={0:"quantity"})
df_user_product.head()

,user_id,product_id,quantity
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [195]:
df_user_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id       int64
product_id    int64
quantity      int64
dtypes: int64(3)
memory usage: 304.6 MB


In [201]:
users = list(np.sort(df_user_product.user_id.unique()))
products = list(df_user_product.product_id.unique())
quantity = list(df_user_product.quantity)

col_indices = df_user_product.user_id.astype('category', categories = users).cat.codes 
row_indices = df_user_product.product_id.astype('category', categories = products).cat.codes 

product_user_matrix = sparse.csr_matrix((quantity, (row_indices, col_indices)), shape=(len(products), len(users)))
product_user_matrix

<49677x206209 sparse matrix of type '<class 'numpy.int64'>'
	with 13307953 stored elements in Compressed Sparse Row format>

In [210]:
# Compute Sparsity
total_size = product_user_matrix.shape[0] * product_user_matrix.shape[1]
actual_size = product_user_matrix.size
sparsity = (1 - (actual_size / total_size)) * 100
sparsity

99.8700882953749

## Fit ALS model

In [213]:
alpha = 40

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

# train the model on a sparse matrix of item/user/confidence weights
model.fit((product_user_matrix * alpha).astype("double"))

# recommend items for a user
# user_items = item_user_data.T.tocsr()
# recommendations = model.recommend(userid, user_items)

# find related items
# related = model.similar_items(itemid)

## Make Test Data

In [293]:
# Read train and test order csvs
df_order_products_train = pd.read_csv("../data/order_products__prior.csv")
df_order_products_submission = pd.read_csv("../data/sample_submission.csv")

In [301]:
current_order_user_df = df_orders.loc[(df_orders.eval_set == "train") | (df_orders.eval_set == "test")]
current_order_user_df = current_order_user_df[["order_id", "user_id"]]
current_order_user_df

,order_id,user_id
10,1187899,1
25,1492625,2
38,2774568,3
44,329954,4
49,2196797,5
53,1528013,6
74,525192,7
78,880375,8
82,1094988,9
88,1822501,10


In [302]:
# Convert train and submission dataframes into the same format
df_order_products_train = df_order_products_train[["order_id", "product_id"]]
df_order_products_train = df_order_products_train.groupby("order_id")["product_id"].apply(list).reset_index()
df_order_products_train.rename(columns={"product_id": "products"})

df_order_products_submission["products"] = df_order_products_submission["products"].apply(lambda p: p.strip().split())

In [303]:
df_order_products_submission.head()

,order_id,products
0,17,"[39276, 29259]"
1,34,"[39276, 29259]"
2,137,"[39276, 29259]"
3,182,"[39276, 29259]"
4,257,"[39276, 29259]"


In [304]:
df_order_products_train.head()

,order_id,product_id
0,2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014..."
1,3,"[33754, 24838, 17704, 21903, 17668, 46667, 174..."
2,4,"[46842, 26434, 39758, 27761, 10054, 21351, 225..."
3,5,"[13176, 15005, 47329, 27966, 23909, 48370, 132..."
4,6,"[40462, 15873, 41897]"


In [306]:
test = set(df_order_products_submission["order_id"].tolist())
train = set(df_order_products_train["order_id"].tolist())
assert len(test & train) == 0
# assert current_order_user_df.shape[0] == (df_order_products_submission.shape[0] + df_order_products_train.shape[0]

In [308]:
df_order_products_submission

,order_id,products
0,17,"[39276, 29259]"
1,34,"[39276, 29259]"
2,137,"[39276, 29259]"
3,182,"[39276, 29259]"
4,257,"[39276, 29259]"
5,313,"[39276, 29259]"
6,353,"[39276, 29259]"
7,386,"[39276, 29259]"
8,414,"[39276, 29259]"
9,418,"[39276, 29259]"


In [ ]:
df_test_user_products = pd.concat([df_order_products_train, df_order_products_submission])

In [ ]:
# If test data csv doesn't exist already, create it
if not Path("../data/test_data.csv").is_file():
    
    